In [1]:
import pandas as pd
pd.set_option("display.max_columns", 500)
import plotly.offline as py
py.init_notebook_mode(connected=True)
from tqdm import tqdm_notebook
import gc
import warnings
warnings.filterwarnings('ignore')

In [2]:
NROWS = None

In [3]:
train_identity = pd.read_csv('../input/train_identity.csv', nrows=NROWS)
train_transaction = pd.read_csv('../input/train_transaction.csv', nrows=NROWS)
train = train_transaction.merge(train_identity, how='left', on='TransactionID')

test_identity = pd.read_csv('../input/test_identity.csv', nrows=NROWS)
test_transaction = pd.read_csv('../input/test_transaction.csv', nrows=NROWS)
test = test_transaction.merge(test_identity, how='left', on='TransactionID')

In [4]:
gc.enable()
del train_identity, train_transaction
del test_identity, test_transaction
gc.collect()

42

In [5]:
print("train.shape:", train.shape)
print("test.shape:", test.shape)

train.shape: (590540, 434)
test.shape: (506691, 433)


In [6]:
target = "isFraud"
test[target] = -1
df = train.append(test)
df.index = range(len(df))

In [572]:
df['uid'] = df["card1"].apply(lambda x: str(x)) + "_" + df["card2"].apply(lambda x: str(x)) +\
                "_" + df["card3"].apply(lambda x: str(x)) + "_" + df["card4"].apply(lambda x: str(x)) +\
                "_" + df["card5"].apply(lambda x: str(x)) + "_" + df["card6"].apply(lambda x: str(x)) +\
                "_" + df["addr1"].apply(lambda x: str(x)) + "_" + df["addr2"].apply(lambda x: str(x)) +\
                "_" + df["P_emaildomain"].apply(lambda x: str(x)) 
# + "_" + df["C1"].apply(lambda x: str(x))
#                 "_" + df["C2"].apply(lambda x: str(x)) + "_" + df["C3"].apply(lambda x: str(x)) +\
#                 "_" + df["C4"].apply(lambda x: str(x)) + "_" + df["C5"].apply(lambda x: str(x)) +\
#                 "_" + df["C6"].apply(lambda x: str(x)) + "_" + df["C7"].apply(lambda x: str(x)) +\
#                 "_" + df["C8"].apply(lambda x: str(x)) + "_" + df["C9"].apply(lambda x: str(x)) +\
#                 "_" + df["C10"].apply(lambda x: str(x)) + "_" + df["C11"].apply(lambda x: str(x)) +\
#                 "_" + df["C12"].apply(lambda x: str(x)) + "_" + df["C13"].apply(lambda x: str(x)) +\
#                 "_" + df["C14"].apply(lambda x: str(x))
                

In [573]:
df.head(3)

,C1,C10,C11,C12,C13,C14,C2,C3,C4,C5,C6,C7,C8,C9,D1,D10,D11,D12,D13,D14,D15,D2,D3,D4,D5,D6,D7,D8,D9,DeviceInfo,DeviceType,M1,M2,M3,M4,M5,M6,M7,M8,M9,P_emaildomain,ProductCD,R_emaildomain,TransactionAmt,TransactionDT,TransactionID,V1,V10,V100,V101,V102,V103,V104,V105,V106,V107,V108,V109,V11,V110,V111,V112,V113,V114,V115,V116,V117,V118,V119,V12,V120,V121,V122,V123,V124,V125,V126,V127,V128,V129,V13,V130,V131,V132,V133,V134,V135,V136,V137,V138,V139,V14,V140,V141,V142,V143,V144,V145,V146,V147,V148,V149,V15,V150,V151,V152,V153,V154,V155,V156,V157,V158,V159,V16,V160,V161,V162,V163,V164,V165,V166,V167,V168,V169,V17,V170,V171,V172,V173,V174,V175,V176,V177,V178,V179,V18,V180,V181,V182,V183,V184,V185,V186,V187,V188,V189,V19,V190,V191,V192,V193,V194,V195,V196,V197,V198,V199,V2,V20,V200,V201,V202,V203,V204,V205,V206,V207,V208,V209,V21,V210,V211,V212,V213,V214,V215,V216,V217,V218,V219,V22,V220,V221,V222,V223,V224,V225,V226,V227,V228,V229,V23,V230,V231,V232,V233,V234,V235,V236,V237,V238,V239,V24,V240,V241,V242,V243,V244,V245,V246,V247,V248,V249,V25,V250,V251,V252,V253,V254,V255,V256,V257,V258,V259,V26,V260,V261,V262,V263,V264,V265,V266,V267,V268,V269,V27,V270,V271,V272,V273,V274,V275,V276,V277,V278,V279,V28,V280,V281,V282,V283,V284,V285,V286,V287,V288,V289,V29,V290,V291,V292,V293,V294,V295,V296,V297,V298,V299,V3,V30,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V31,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V32,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V33,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339,V34,V35,V36,V37,V38,V39,V4,V40,V41,V42,V43,V44,V45,V46,V47,V48,V49,V5,V50,V51,V52,V53,V54,V55,V56,V57,V58,V59,V6,V60,V61,V62,V63,V64,V65,V66,V67,V68,V69,V7,V70,V71,V72,V73,V74,V75,V76,V77,V78,V79,V8,V80,V81,V82,V83,V84,V85,V86,V87,V88,V89,V9,V90,V91,V92,V93,V94,V95,V96,V97,V98,V99,addr1,addr2,card1,card2,card3,card4,card5,card6,dist1,dist2,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,isFraud,uid,day
0,1.0,0.0,2.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,14.0,13.0,13.0,NaN,NaN,NaN,0.0,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,T,T,T,M2,F,T,NaN,NaN,NaN,NaN,W,NaN,68.5,86400,2987000,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,117.0,0.0,0.0,1.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,315.0,87.0,13926,NaN,150.0,discover,142.0,credit,19.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,13926_nan_150.0_discover_142.0_credit_315.0_87...,1
1,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Na

In [574]:
df["day"] = (df["TransactionDT"] + 3600 * 12) // (24 * 60 * 60)

In [583]:
feature_list = ["uid", target, "D1", "D10", "day", "TransactionDT", "TransactionID"]

In [584]:
train = df.loc[df[target] != -1, feature_list]

In [585]:
train["uid"].nunique()

94922

In [586]:
temp = train.groupby('uid').agg({target: ['mean', 'count', 'sum']})
# temp['uid'] = temp.index
# temp.index = range(len(temp))
# temp.columns = ['uid', 'mean', 'count', 'sum']

In [587]:
uid_list = list(temp.loc[(temp[target]['mean'] ==1) & (temp[target]['sum'] > 0)].index)

In [588]:
len(uid_list)

1801

In [589]:
sub1 = pd.read_csv('../output/ens9.csv')
sub1.shape

(506691, 2)

In [590]:
sub1 = sub1.merge(df[feature_list], on = "TransactionID", how = "left")

In [591]:
sub1.head()

,TransactionID,isFraud_x,uid,isFraud_y,D1,D10,day,TransactionDT
0,3663549,0.000715,10409_111.0_150.0_visa_226.0_debit_170.0_87.0_...,-1,419.0,418.0,213,18403224
1,3663550,0.001111,4272_111.0_150.0_visa_226.0_debit_299.0_87.0_a...,-1,149.0,231.0,213,18403263
2,3663551,0.000770,4476_574.0_150.0_visa_226.0_debit_472.0_87.0_h...,-1,137.0,136.0,213,18403310
3,3663552,0.000563,10989_360.0_150.0_visa_166.0_debit_205.0_87.0_...,-1,42.0,242.0,213,18403310
4,3663553,0.001380,18018_452.0_150.0_mastercard_117.0_debit_264.0...,-1,22.0,22.0,213,18403317


In [593]:
sub1["last_day"] = sub1["day"] - sub1["D1"]

In [594]:
subx = sub1.loc[(sub1["uid"].isin(uid_list)) & (sub1["last_day"] <= 183) & (sub1["last_day"] >= 1)]
subx

,TransactionID,isFraud_x,uid,isFraud_y,D1,D10,day,TransactionDT,last_day
9745,3673294,0.147820,5993_555.0_150.0_visa_226.0_credit_181.0_87.0_...,-1,114.0,448.0,217,18714409,103.0
26331,3689880,0.508255,9002_453.0_150.0_visa_226.0_debit_264.0_87.0_h...,-1,173.0,173.0,223,19284073,50.0
27659,3691208,0.951876,13623_585.0_150.0_visa_226.0_debit_498.0_87.0_...,-1,106.0,106.0,224,19338539,118.0
29268,3692817,0.839758,13994_127.0_150.0_mastercard_224.0_debit_269.0...,-1,125.0,58.0,225,19404771,100.0
30251,3693800,0.962166,13623_585.0_150.0_visa_226.0_debit_498.0_87.0_...,-1,107.0,107.0,225,19427257,118.0
32301,3695850,0.968290,13623_585.0_150.0_visa_226.0_debit_498.0_87.0_...,-1,108.0,108.0,226,19501604,118.0
35923,3699472,0.878980,13623_585.0_150.0_visa_226.0_debit_498.0_87.0_...,-1,109.0,109.0,227,19604901,118.0
37380,3700929,0.863976,13623_585.0_150.0_visa_226.0_debit_498.0_87.0_...,-1,110.0,110.0,228,19677179,118.0
37404,3700953,0.938172,13623_585.0_150.0_visa_226.0_debit_498.0_87.0_...,-1,110.0,164.0,228,19677652,118.0
39551,3703100,0.944614,13623_585.0_150.0_visa_226.0_debit_498.0_87.0_...,-1,111.0,165.0,229,19755994,118.0


In [607]:
i = 92

In [608]:
list(subx["uid"])[i]

'4383_271.0_150.0_visa_226.0_credit_324.0_87.0_aol.com'

In [609]:
list(subx["last_day"])[i]

183.0

In [610]:
train.loc[(train["uid"] == list(subx["uid"])[i])]

,uid,isFraud,D1,D10,day,TransactionDT,TransactionID
589092,4383_271.0_150.0_visa_226.0_credit_324.0_87.0_...,1,0.0,45.0,183,15781904,3576092
589166,4383_271.0_150.0_visa_226.0_credit_324.0_87.0_...,1,0.0,45.0,183,15783628,3576166
590350,4383_271.0_150.0_visa_226.0_credit_324.0_87.0_...,1,0.0,45.0,183,15807118,3577350


In [132]:
fraud_Uids = []

In [133]:
from tqdm import tqdm_notebook
for uid in tqdm_notebook(train["uid"].unique()[-3000:]):
    
    temp1 = df.loc[(df["uid"] == uid) & (df[target] == 1), feature_list]
    
    temp2 = df.loc[(df["uid"] == uid) & (df[target] == 0), feature_list]
    
    temp3 = df.loc[(df["uid"] == uid) & (df[target] == -1), feature_list]
    
    if (temp1.TransactionID.min() > temp2.TransactionID.max() and temp3.shape[0] > 0):
        
        print(uid, temp3.shape[0])
        fraud_Uids.append([uid, temp3.shape[0]])

In [111]:
fraud_Uids = pd.DataFrame(fraud_Uids)

In [112]:
fraud_Uids.columns = ['uid', 'train_cnt', 'test_cnt']
fraud_Uids

,uid,train_cnt,test_cnt
0,11839_490.0_150.0_visa_226.0_debit_280.0_420.0...,1,1
1,11839_490.0_150.0_visa_226.0_debit_117.0_420.0...,5,3
2,12786_399.0_150.0_american express_102.0_credi...,2,1


In [114]:
print(fraud_Uids["test_cnt"].sum())

5


In [115]:
fraud_Uids.to_csv("./fraud_Uids.csv", index=False)

In [116]:
fraud_Uids_rule = list(fraud_Uids.uid)

In [117]:
fraud_Uids_rule

['11839_490.0_150.0_visa_226.0_debit_280.0_420.0_87.0_aol.com',
 '11839_490.0_150.0_visa_226.0_debit_117.0_420.0_87.0_aol.com',
 '12786_399.0_150.0_american express_102.0_credit_225.0_325.0_87.0_gmail.com']

- todo: 不同的uid, D系列

## 使用规则进行后处理

In [240]:
sub1 = pd.read_csv('../output/ens9.csv')
sub1.shape

(506691, 2)

In [241]:
sub1.head(3)

,TransactionID,isFraud
0,3663549,0.000715
1,3663550,0.001111
2,3663551,0.000770


In [242]:
sub1 = sub1.merge(df[["TransactionID", "uid"]], on = "TransactionID", how = "left")

In [243]:
sub1.head(3)

,TransactionID,isFraud,uid
0,3663549,0.000715,10409_111.0_150.0_visa_226.0_debit_170.0_87.0_...
1,3663550,0.001111,4272_111.0_150.0_visa_226.0_debit_299.0_87.0_a...
2,3663551,0.000770,4476_574.0_150.0_visa_226.0_debit_472.0_87.0_h...


In [248]:
sub1.loc[sub1["uid"].isin(uid_list), "isFraud"] = 1

In [249]:
sub1[["TransactionID", "isFraud"]].to_csv("../output/ens9_rule3.csv", index=False)